In [2]:
import pyodbc
import pandas as pd
import os
import itertools

## Connect to as400

In [9]:
cnxn = pyodbc.connect('DRIVER={Client Access ODBC Driver (32-bit)};SIGNON=1;REMARKS=1;LIBVIEW=1;DFT=1;QRYSTGLMT=-1;PKG=CLPB80F/DEFAULT(IBM),2,0,1,0,512;LANGUAGEID=ENU;DFTPKGLIB=B833CLPF;DBQ=B833CLPF;SYSTEM=NAMFG401;UserID=CLSERVICE;Password=SERVICE;')
cursor = cnxn.cursor()

## Shop Order Queries

### archived_SO.csv
Completed Shop Orders for training

In [18]:
test_query = '''
SELECT *
FROM B833CLPF.FMA'''

In [20]:
cursor.execute(test_query)
row = cursor.fetchone()


In [21]:
row.MORD

Decimal('129629')

In [10]:
archived_so_query = '''
SELECT DISTINCT FMH.MORD, FMH.MPROD, FMH.MRDTE, FMH.MQISS, FMH.MAPRD, FMH.MASTS, FMH.MBOM, FMH.MOPNO, FSH.SQFIN, FMH.MQREQ, FSH.SQREQ, FSH.SOLOT
FROM B833CLPF.FMH FMH, B833CLPF.FSH FSH
WHERE FMH.MORD = FSH.SORD AND ((FMH.MRDTE>20191231) AND (FSH.SQFIN>0))'''
queue_so_query = '''SELECT DISTINCT FMA.MORD, FMA.MPROD, FMA.MRDTE, FMA.MQREQ, FMA.MQISS, FMA.MAPRD, FMA.MBOM, FMA.MOPNO, FSO.SQFIN, FSO.SQREQ, FSO.SOLOT, FSO.SOSTS
FROM B833CLPF.FMA FMA, B833CLPF.FSO FSO, B833CLPF.MBM MBM
WHERE FMA.MORD = FSO.SORD AND FSO.SPROD = MBM.BPROD AND ((FMA.MRDTE>20210901) AND (FSO.SQFIN=0))'''



In [11]:
%%time
df = pd.DataFrame()
for i, row in enumerate(itertools.islice(cursor.execute(archived_so_query),100000)):
    df.loc[i,'shop_order'] = row.MORD
    df.loc[i,'lot_number'] = row.SOLOT
    df.loc[i,'Product'] = row.MAPRD
    df.loc[i,'Date'] = row.MRDTE
    df.loc[i,'component'] = row.MPROD
    df.loc[i,'BOM_qty'] = row.MBOM
    df.loc[i,'MOPNO'] = row.MOPNO
    df.loc[i,'requested_qty'] = row.SQREQ
    df.loc[i,'finished_qty'] = row.SQFIN
    df.loc[i,'MQISS'] = row.MQISS
    df.loc[i,'MQREQ'] = row.MQREQ
df.head()
df.to_csv('./raw_data/archived_so.csv')

Wall time: 21min 40s


In [8]:
%%time
cursor.execute(archived_so_query)
rows = cursor.fetchall()
df = pd.DataFrame.from_records(rows,columns = ['shop_order','component','date','MQREQ','product','MQISS','bom_qty','MOPNO','finished_qty','SQREQ','SOSTS','lot_number'])
df.head()

Wall time: 5.02 s


,shop_order,component,date,MQREQ,product,MQISS,bom_qty,MOPNO,finished_qty,SQREQ,SOSTS,lot_number
0,190875,EXP5013,20200203,0,5551761207,05,0,0,36000.000,0,36000.000,19L13263X
1,190875,EXP6001,20200203,3.996,5551761207,05,0.000111,888,36000.000,3.996,36000.000,19L13263X
2,190875,FS10013527,20200203,0,5551761207,05,0,0,36000.000,0,36000.000,19L13263X
3,190875,4101801,20200203,36000.000,5551761207,05,1.000000,888,36000.000,36000.000,36000.000,19L13263X
4,189714,FS10002899,20200102,0,WP8985,05,0,0,23929.000,0,23929.000,19H189714


### queue_so
shop orders waiting to go to production

In [4]:
%%time
df = pd.DataFrame()
for i, row in enumerate(itertools.islice(cursor.execute(queue_so_query),100000)):
    df.loc[i,'shop_order'] = row.MORD
    df.loc[i,'lot_number'] = row.SOLOT
    df.loc[i,'Product'] = row.MAPRD
    df.loc[i,'Date'] = row.MRDTE
    df.loc[i,'component'] = row.MPROD
    df.loc[i,'BOM_qty'] = row.MBOM
    df.loc[i,'MOPNO'] = row.MOPNO
    df.loc[i,'requested_qty'] = row.SQREQ
    df.loc[i,'finished_qty'] = row.SQFIN
    df.loc[i,'MQISS'] = row.MQISS
    df.loc[i,'MQREQ'] = row.MQREQ
df.head()
df.to_csv('./raw_data/queue_so.csv')

Wall time: 8 s


### current_so
shop orders currently in production

## NCs as-400 Queries (not currently working)

In [86]:
'nc_test_query not working, see copynpast_of_NCR_TESTsql.txt for og source'
stardate = "20210901"
endte = "20211001"
nc_test_query = f'''SELECT Distinct 
chncr#.nr4nchp EDTCDE(L), chdsdt.nr4nchp,COLHDG("NCR Open" "Date") EDTCDE(W), chdvdt.nr4nchp COLHDG("NCR Closed " "Date") EDTCDE(W), 
chdvdt-CHDSDT COLHDG("NCR Cycle Time"), chtype.nr4nchp, chsts.nr4nchp, chassd.nr4nchp COLHDG("Shop Order"), chendt.nr4nchp COLHDG("BPCS Entry Date") EDTCDE(W), 
chcldt.nr4nchp COLHDG("BPCS Closure") EDTCDE(W), chinvs.nr4nchp COLHDG("Inititated" "By"), chidpt.nr4nchp COLHDG("Area/Line"), chodpt.nr4nchp COLHDG("Person" "Responsible"), 
chrcv5.nr4nchp COLHDG("Containment" "Code"), chvend.nr4nchp, vndnam.avm, chscar.nr4nchp, chltsz.nr4nchp COLHDG("NCR" "Qty"), chsmsz.nr4nchp COLHDG("Sample" "Qty"), 
chdeft.nr4nchp COLHDG("Defect" "Qty"), chrewk.nr4nchp COLHDG("Rework" "Instructions"), chusr1.nr4nchp COLHDG("Source/Type" "Defined1"), chusr2.nr4nchp COLHDG("Nonconformance" "Category"), 
chpp#.nr4nchp, chitmc.nr4nchp, chplt#.nr4nchp, chpuom.nr4nchp, chpflmc1.nr4nchp COLHDG("Parent" "Failure" "Mode"), ccdesc.zcc, ccsdsc.zcc, chprtcc1.nr4nchp COLHDG("Parent" "Root Cause/" "Location"), 
chprtvq.nr4nchp COLHDG("Parent " "Return to Vendor"  "Quantity"), chpuaiq.nr4nchp COLHDG("Parent" "Rework" "Quantity"), chpothq.nr4nchp COLHDG("Parent" "Sort Good" "Quantity"), 
chpaecq.nr4nchp COLHDG("Parent" "Use As Is" "Quantity"), chpagdq.nr4nchp COLHDG("Parent" "Scrap" "Quantity"), chprskq.nr4nchp COLHDG("Parent" "Sort Bad" "Quantity"), 
chpscpq.nr4nchp COLHDG("Parent" "N/A #1" "Quantity"), chprewq.nr4nchp COLHDG("Parent" "N/A #2" "Quantity"), chpsrtq.nr4nchp COLHDG("Parent" "N/A #3" "Quantity"), 
chpchar.nr4nchp COLHDG("Parent Item" "Confirmed Defect" "Description"), chpspec.nr4nchp COLHDG("Parent Item" "Cause"), chpnonc.nr4nchp COLHDG("Parent" "Correction" "Corrective  Action"), 
chcrtby.nr4nchp, chcrtdt.nr4nchp EDTCDE(W), cdseq#.nr4ncdp, cdcp#.nr4ncdp, cdcuom.nr4ncdp, cdcflmc1.nr4ncdp COLHDG("Component" "Failure Mode"), cdcrtcc1.nr4ncdp COLHDG("Component" "Root Cause"), 
cdcrtvq.nr4ncdp COLHDG("Component" "Return to Vendor"  "Quantity"), cdcuaiq.nr4ncdp COLHDG("Component" "Rework" "Quantity"), cdcothq.nr4ncdp COLHDG("Component" "Sort Good" "Quantity"), 
cdcaecq.nr4ncdp COLHDG("Component" "Use As Is" "Quantity"), cdcagdq.nr4ncdp COLHDG("Component" "Scrap" "Quantity"), cdcrskq.nr4ncdp COLHDG("Component" "Sort Bad" "Quantity"), cdcscpq.nr4ncdp COLHDG("Component" "N/A #1" "Quantity"), 
cdcrewq.nr4ncdp COLHDG("Component" "N/A #2" "Quantity"), cdcsrtq.nr4ncdp COLHDG("Component" "N/A #3" "Quantity"), cdcchar.nr4ncdp COLHDG("Component" "Confirmed Defect"  "Description"), cdcspec.nr4ncdp COLHDG("Component" "Cause"), 
cdcnonc.nr4ncdp COLHDG("Component" "Correction /" "Corrective  Action")
FROM  b833clpuf/nr4nchp,b833clpuf/nr4nltp,b833clpuf/nr4ncdp,b833clpf/zcc,b833clpf/avm
PARTIAL OUTER JOIN  ltncr#.nr4nltp=chncr#.nr4nchp and ltncr#.nr4nltp=cdncr#.nr4ncdp and chpflmc1.nr4nchp=cccode.zcc
and chvend.nr4nchp=vendor.avm
WHERE chdsdt BETWEEN {stardate} AND {endte} and chsts<>"V'''

nc_test_query_ii = f'''SELECT Distinct 
chncr#.nr4nchp, chdsdt.nr4nchp, chdvdt.nr4nchp, 
chdvdt-CHDSDT, chtype.nr4nchp, chsts.nr4nchp, chassd.nr4nchp, chendt.nr4nchp,
chcldt.nr4nchp, chinvs.nr4nchp, chidpt.nr4nchp, chodpt.nr4nchp, 
chrcv5.nr4nchp, chvend.nr4nchp, vndnam.avm, chscar.nr4nchp, chltsz.nr4nchp, chsmsz.nr4nchp, 
chdeft.nr4nchp, chrewk.nr4nchp , chusr1.nr4nchp, chusr2.nr4nchp, 
chpp#.nr4nchp, chitmc.nr4nchp, chplt#.nr4nchp, chpuom.nr4nchp, chpflmc1.nr4nchp, ccdesc.zcc, ccsdsc.zcc, chprtcc1.nr4nchp, 
chprtvq.nr4nchp, chpuaiq.nr4nchp, chpothq.nr4nchp, chpaecq.nr4nchp, chpagdq.nr4nchp, chprskq.nr4nchp, chpscpq.nr4nchp, chprewq.nr4nchp, chpsrtq.nr4nchp, 
chpchar.nr4nchp, chpspec.nr4nchp, chpnonc.nr4nchp, 
chcrtby.nr4nchp, chcrtdt.nr4nchp, cdseq#.nr4ncdp, cdcp#.nr4ncdp, cdcuom.nr4ncdp, cdcflmc1.nr4ncdp, cdcrtcc1.nr4ncdp, 
cdcrtvq.nr4ncdp, cdcuaiq.nr4ncdp, cdcothq.nr4ncdp, 
cdcaecq.nr4ncdp, cdcagdq.nr4ncdp, cdcrskq.nr4ncdp, cdcscpq.nr4ncdp, 
cdcrewq.nr4ncdp, cdcsrtq.nr4ncdp, cdcchar.nr4ncdp, cdcspec.nr4ncdp, cdcnonc.nr4ncdp
FROM  b833clpuf/nr4nchp,b833clpuf/nr4nltp,b833clpuf/nr4ncdp,b833clpf/zcc,b833clpf/avm
PARTIAL OUTER JOIN  ltncr#.nr4nltp=chncr#.nr4nchp and ltncr#.nr4nltp=cdncr#.nr4ncdp and chpflmc1.nr4nchp=cccode.zcc
and chvend.nr4nchp=vendor.avm
WHERE chdsdt BETWEEN {stardate} AND {endte} and chsts<>"V'''



In [89]:
cursor.execute(nc_test_query_ii)

Error: ('HY000', '[HY000] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL5016 - Qualified object name NR4NCHP not valid. (-5016) (SQLExecDirectW); [HY000] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL7973 - SQL create package for PYTHONARBA in B833CLPF has failed. (-551)')

## QMSPRODRP query



In [13]:

qmsprodrp_query = ''' SELECT       	ttype.ithl01, CVTDATE(TTDTE,ymd1) NAME(TRANDATE) 
				COLHDG("Trans" "Date"), 
				thord.ithl01 COLHDG("Shop Order" "Number") EDTCDE(4), 
				iityp.iiml01 COLHDG("Item" "Type"), iclas.iiml01, tprod.ithl01, 
				idesc.iiml01, twhs.ithl01 COLHDG(" " "Warehouse"), 
				tloct.ithl01 COLHDG(" " "Location"), tlot.ithl01, sum(tqty) 
				NAME(SUMTQTY) COLHDG("Transaction" "Quantity") 
				EDTCDE(J), 
				iums.iiml01, iump.iiml01
 FROM          	ithl01,iiml01
 INNER JOIN  	tprod.ithl01=iprod.iiml01
 WHERE        	trandate BETWEEN "&&stardate" AND "&&enddte" 
				and thfac="63" AND &Type=TTYPE
 GROUP BY  	ttype,TRANDATE,thord,iityp,iclas,tprod,idesc,twhs,tloct,tlot,
				iums,iump
 ORDER BY  	tprod Asc,thord Asc'''

In [91]:
exp_search = """
SELECT * FROM IASP.LIB.FILE
"""
cursor.execute(exp_search)

Error: ('HY000', '[HY000] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL0950 - Relational database IASP not in relational database directory. (-950) (SQLExecDirectW)')

In [39]:
table_search = '''
SELECT
    *
FROM
    information_schema.tables'''
cursor.execute(table_search)

In [40]:
%%time
df = pd.DataFrame(itertools.islice(cursor.execute(table_search),100000))
df.to_csv('table_list.csv')

Wall time: 46.1 s


In [41]:
col_search = '''
SELECT
    *
FROM
    information_schema.columns'''
cursor.execute(col_search)

In [42]:
%%time
df = pd.DataFrame(itertools.islice(cursor.execute(col_search),10000))
df.to_csv('col_list.csv')

Wall time: 52.4 s


In [49]:
within_table_search = '''
SELECT
    *
FROM
    B833CLPF'''
cursor.execute(within_table_search)

ProgrammingError: ('42S02', '[42S02] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL0204 - B833CLPF in B833CLPF type *FILE not found. (-204) (SQLExecDirectW)')

In [45]:
cursor.fetchone()
cursor.statistics('B833CLPF')

ProgrammingError: No results.  Previous SQL was not a query.

In [81]:

test = pd.DataFrame(columns = cols)
test.loc[0,:] = list(row)

In [82]:
test

,1,2,3,4,5,6,7,8,9,10
0,NAMFG401,QSYS,QACSTOSDS,BASE TABLE,None,None,None,None,None,YES


In [5]:
row = cursor.fetchone()

In [63]:
rows = cursor.fetchall()

In [59]:
cursor.tables()

OperationalError: ('08S01', '[08S01] [IBM][System i Access ODBC Driver]Communication link failure. comm rc=10054 - CWBCO1047 - The IBM i server application  disconnected the connection (10054) (SQLTables)')

In [53]:
cursor.tables()

## Plantstar Query

doesn't work with 64-bit python

In [7]:
ps_cnxn = pyodbc.connect("Provider=MSDASQL.1;Persist Security Info=True;Extended Properties=&quot;DSN=mypstar;&quot;;Initial Catalog=focus2000")
ps_cursor = ps_cnxn.cursor()

Error: ('IM014', '[IM014] [Microsoft][ODBC Driver Manager] The specified DSN contains an architecture mismatch between the Driver and Application (0) (SQLDriverConnect)')

In [ ]:
plant_query = '''SELECT ss_hist_base_0.start_time, ss_hist_base_0.mach_name, ss_hist_base_0.gross_pieces, ss_hist_base_0.user_text_1, ss_hist_base_0.user_text_4, ss_hist_base_0.mat_formula, ss_hist_base_0.tool, ss_hist_base_0.std_shot_weight, ss_hist_base_0.act_shot_weight, ss_hist_base_0.start_time
FROM focus2000.dg_def dg_def_0, focus2000.dg_set dg_set_0, focus2000.ss_hist_base ss_hist_base_0
WHERE dg_def_0.mach_seq = ss_hist_base_0.mach_seq AND dg_set_0.dg_seq = dg_def_0.dg_seq AND ((dg_set_0.dg_name Like 'Team%') AND (ss_hist_base_0.start_time>{ts '2019-03-01 06:00:00'}))'''